### Tratamento e Limpeza dos Dataframes
---

### Stock Market Data Cleaning Pipeline  

Pipeline de limpeza e preparação de dados para análise financeira do dataset `stock_part1.csv`.

---

#### **Principais Etapas**  

1. **Datas**  
   - Conversão para formato `datetime` com reconhecimento automático de padrões.  
   - Identificação de 1.015% de datas inválidas (convertidas para `NaT`).  
   - Correção de 498 registros com datas futuras (troca de dia/mês).  

2. **Tickers**  
   - Padronização em caixa alta e remoção de espaços.  
   - Correção de tickers com erros via *fuzzy matching* (limiar: 65% de similaridade).  
   - tickers que não obtiveram semelhança suficiente foram **eliminados**.  

3. **Colunas Numéricas**  
   - Remoção de caracteres especiais (`$`, `Z→2`, `,`→`.`).  
   - Conversão para `float` em: `Open`, `High`, `Low`, `Close`, `Adj Close`, `Dividend`.  
   - Reordenação automática de valores para garantir `High ≥ Low`.  

4. **Volume**  
   - Padronização de unidades (`K`→×10³, `M`→×10⁶, `B`→×10⁹).  
   - Conversão para inteiro (`int64`).  

5. **Split Ratio (Coluna eliminada)**
   - Conjunto de formatos `1:1`, `2:1`, `3:2`.
  - Identificação de 25.76% de valores inválidos.  
  - **Após análise, considerando a relevancia da variável, a coluna foi eliminada**

In [1]:
%pip install fuzzywuzzy
%pip install Levenshtein
%pip install -U nbformat
%pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.5 MB/s eta 0:00:00


**Importando as bibliotecas necessárias**

- pandas: utilizado para a manipulação de dataframes.
- numpy: usado para realizar operações matemáticas mais complexas e vetorizadas.
- fuzzywuzzy: permite quantificar a semelhança entre palavras ou frases.
- plotly.express: utilizado para a visualização dos dados de forma interativa.
- functools: fornece funções auxiliares de ordem superior, como `reduce` e `partial`.
- datetime: tratamento de datas.

In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import plotly.express as px
from functools import lru_cache
from datetime import datetime

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df1 = pd.read_csv('/content/drive/MyDrive/Projeto Dados CITI (Squad Dani)/Databases/stock_part1.csv')

1.  **Tratamento de Datas**

- Datas inconsistentes foram convertidas para um valor padrão `NaT` (Not a Time).
- Uma coluna auxiliar chamada `"original_date"` foi criada para armazenar os valores originais das datas antes da conversão, auxiliando na rastreabilidade.   
- Datas que não puderam ser padronizadas foram contabilizadas para revisão subsequente e potencial tratamento manual.

In [5]:
def process_date_column(df, column_name):


    # Converte a coluna para datetime
    df[column_name] = pd.to_datetime(df[column_name], format="mixed", errors='coerce')

    # Identifica datas inválidas
    invalid_dates = df[df[column_name].isna()]
    error_count = len(invalid_dates)
    error_percent = error_count / len(df[column_name])

    print(f"Percentage of errors in '{column_name}': {error_percent * 100:.2f}%")

    # Datas no futuro
    future_dates = df[df[column_name] > datetime.now()][column_name]
    df[column_name] = df[column_name].apply(lambda x: x.replace(tzinfo=None) if pd.notnull(x) else x)


    return df, invalid_dates, future_dates, error_percent

df1, invalid_dates, future_dates, error_percent = process_date_column(df1, 'Date')

Percentage of errors in 'Date': 1.01%


Teste para identificar datas que foram erroneamente interpretadas como datas futuras.

* Observando o padrão dessas datas futuras, nota-se que os valores de dia e mês parecem estar trocados.
* Isso indica um problema de formatação que necessita de correção para padronizar estas datas acuradamente.

In [6]:
maiormes = 0
maiordia = 0
# Itera sobre as datas futuras para identificar se a troca de dia/mês é um padrão consistente
for i in future_dates.dt.strftime('%Y-%m-%d'):
    diaatual = int(i[5:7])
    mesatual = int(i[-2:])
    maiordia = ( diaatual if diaatual > diaatual else diaatual)
    maiormes = ( mesatual if mesatual > maiormes else maiormes)

print(maiormes)
print(maiordia)

5
8


In [7]:
future_dates = df1[df1["Date"] > datetime.now()]["Date"]
print(future_dates)

715      2025-11-02
948      2025-09-05
1036     2025-07-03
2830     2025-09-05
2880     2025-10-01
            ...    
197447   2025-09-03
197677   2025-10-03
198021   2025-09-05
198338   2025-09-05
199004   2025-08-04
Name: Date, Length: 420, dtype: datetime64[ns]


* Transformação das datas futuras para o seu formato correto através da troca de dia e mês.
* Remoção de informações de fuso horário (timezone), pois não estavam presentes no DataFrame original e não são necessárias.
* A coluna temporária `"original_date"` é descartada após a conclusão do processamento.

In [8]:
future_dates = df1[df1['Date'] > datetime.now()]['Date']
# Para datas identificadas como futuras, troca o dia e o mês
df1.loc[future_dates.index, 'Date'] = future_dates.apply(lambda x: x.replace(day=x.month, month=x.day))
# Remove informações de fuso horário


2.  **Processamento de Tickers (Símbolos de Ações)**
    * Início do processo de limpeza e padronização dos símbolos de tickers.
    * Padronização básica inclui a conversão dos tickers para caixa alta e a remoção de espaços em branco.
    * Tickers são considerados potencialmente válidos para correção se aparecem mais de 25 vezes no dataset, formando uma lista base de `valid_tickers`.

In [9]:
ticker_counts = df1['Ticker'].value_counts()
df1["Ticker"] = df1["Ticker"].str.upper().str.strip() # Converte tickers para maiúsculas e remove espaços
ticker_counts = df1["Ticker"].value_counts()
valid_tickers = list(set(
    ticker_counts[ticker_counts > 25].index.tolist()
))
valid_tickers = [t.upper() for t in valid_tickers]

Função para encontrar a melhor correspondência aproximada (fuzzy match) para um dado ticker a partir de uma lista de candidatos.

* Uma correspondência é considerada válida se seu score de similaridade for de 65% ou mais.
* Filtra candidatos por tamanho para melhorar a precisão e eficiência da correspondência.

In [10]:
def get_fuzzy_match(ticker, candidates, min_score=65):
    if not ticker or len(ticker) < 2: # Pula tickers muito curtos ou vazios
        return None
    # Filtra candidatos para aqueles com tamanho similar ao ticker (±1 caractere)
    candidates_filtered = [t for t in candidates if abs(len(t) - len(ticker)) <= 1]
    if not candidates_filtered:
        return None
    match, score = process.extractOne(ticker, candidates_filtered) # Encontra a melhor correspondência
    if score >= min_score: # Se o score atingir o limiar mínimo, retorna a correspondência
        return match
    return None

Um dicionário `corrections` é criado para armazenar mapeamentos de tickers incorretos para corrigidos.
O processo envolve:
1. Iterar por tickers que aparecem com baixa frequência (<= 25 vezes).
3. Se uma correspondência aproximada é encontrada, ela é adicionada a `corrections`.
4. Se não houver correspondência aproximada, a linha é eliminada

In [11]:
corrections = {}
rare_valid = 0
for ticker in ticker_counts[ticker_counts <= 25].index:
  corrected = get_fuzzy_match(ticker, valid_tickers)
  if corrected:
    corrections[ticker] = corrected
  else:
    df1.drop(df1[df1["Ticker"] == ticker].index, inplace=True)


df1["Ticker"] = df1["Ticker"].replace(corrections)
print("corrections:", len(corrections))

corrections: 2574


3.  **Processamento de Colunas Numéricas**
* Análise inicial das colunas numéricas (`Open`, `High`, `Low`, `Close`, `Adj Close`, `Volume`, `Dividend`).
* Observa-se que todas as colunas, exceto "Adj Close", exibem um padrão similar de ruído (caracteres não numéricos).

In [12]:
numerics_columns = ["Open", "High", "Low", "Close", "Adj Close", "Volume", "Dividend"]
# Função para identificar todos os caracteres não numéricos únicos em uma coluna
def get_non_numeric_characters(column):
  print(column.name)
  tot = set("")
  for index, value in column.items():
    tot = tot | set([c for c in value if not c.isdigit()])
  print(tot)

# Função para contar quantos valores string em uma coluna possuem vírgula ou ponto como penúltimo ou antepenúltimo caractere
def count_comma(column):
    count = 0
    for index,value in column.items():
        if isinstance(value, str) and len(value) >= 3:
            if value[-3] in [',', '.']:
                count += 1
    print(count)

for i in numerics_columns:
  get_non_numeric_characters(df1[i])
  count_comma(df1[i])

Open
{'$', ',', '.', 'Z'}
200000
High
{'$', ',', '.', 'Z'}
200000
Low
{'$', ',', '.', 'Z'}
200000
Close
{'$', ',', '.', 'Z'}
200000
Adj Close
{'$', ',', '.', 'Z'}
200000
Volume
{'o', ' ', 'n', 'K', 'i', 'b', 'l', ',', '.', 'M', 'B'}
3503
Dividend
{'$', ',', '.', 'Z'}
200000


- A coluna "Volume" é temporariamente removida da lista de colunas numéricas para tratamento individualizado e especializado.
- Caracteres problemáticos (ex: `$`, `Z`) são limpos das colunas numéricas restantes.
- Pontos são inicialmente trocados por vírgulas, provavelmente em preparação para um formato específico de conversão decimal.

In [13]:
equivalents_numeric_columns = numerics_columns[:]
equivalents_numeric_columns.remove("Volume") # Remove "Volume" para processamento separado

def clean_and_convert_number(value):
    # Remove espaços, símbolos monetários e substitui caracteres problemáticos
    value = value.strip().strip("$").replace("Z", "2").replace(".", ",")

    # Substitui a última vírgula por um ponto para formato decimal
    if len(value) >= 3:
        value = value[:-3] + "." + value[-2:]

    try:
        return float(value)
    except:
        return None  # ou np.nan se preferir trabalhar com pandas

for col in equivalents_numeric_columns:
    df1[col] = df1[col].apply(clean_and_convert_number)

Padroniza a representação decimal substituindo a vírgula (agora na posição decimal típica) por um ponto, e então converte para float.
Este passo assume que números como '1,234,56' foram transformados para '1234,56' e agora se tornam '1234.56'.

Garante consistência nos dados de precificação: `High` deve ser sempre o máximo e `Low` o mínimo entre `Open`, `High`, `Low`, `Close`.
* A coluna "Adj Close" não é incluída neste ajuste específico porque, em casos especiais (como desdobramentos reversos), ela pode legitimamente ser maior que "High" ou menor que "Low".

In [14]:
def corrigir_high_low(df):

    df_corrigido = df.copy()
    cols = ["Open", "High", "Low", "Close"]
    valores = df_corrigido[cols].values
    min_val = np.min(valores, axis=1) # Encontra o valor mínimo por linha
    max_val = np.max(valores, axis=1) # Encontra o valor máximo por linha
    resto_valores = []

    for linha, menor, maior in zip(valores, min_val, max_val):
        restante = [v for v in linha if v != menor and v != maior]

        if len(restante) < 2: # Lida com casos com valores duplicados
            linha_temp = list(linha)
            linha_temp.remove(menor)
            linha_temp.remove(maior)
            restante = linha_temp

        resto_valores.append(restante)

    open_new = [v[0] for v in resto_valores]
    close_new = [v[1] for v in resto_valores]

    df_corrigido["High"] = max_val # Atribui o máximo verdadeiro a 'High'
    df_corrigido["Low"] = min_val # Atribui o mínimo verdadeiro a 'Low'
    df_corrigido["Open"] = open_new
    df_corrigido["Close"] = close_new
    return df_corrigido


df1 = corrigir_high_low(df1) # Aplica a correção ao DataFrame

Em alguns casos, os valores de "Volume" ficam com "billion" ao lado, mesmo todas elas ja estando em bilhões (ex: 2371245326 Billion). Dessa forma é necessaria a remoção desses valores irrelevantes.

In [15]:
get_non_numeric_characters(df1["Volume"].apply( lambda x: x.strip())) # Assume que Volume é string aqui.

# Remove espaços em branco e caracteres das palavras "billion" de cada string na coluna "Volume"
def remove_billion_chars(s):
    return ''.join(c for c in s.strip() if c.lower() not in 'billion')

df1["Volume"] = df1["Volume"].apply(remove_billion_chars)

Volume
{'o', ' ', 'n', 'K', 'i', 'b', 'l', ',', '.', 'M', 'B'}


Conversão correta em valores que tem M e ou K junto aos números
Numeros como 4.282.547.185K não são multiplicados por 10**3 pois já estão na casa dos bilhões

In [16]:
# Função para remover 'M', 'K' e padronizar separadores decimais/milhares para conversão para float
def remove_letter(value):
  value = value.replace("M", "")
  value = value.replace("K", "")
  if value.count(",") > 1: # Se múltiplas vírgulas, assume que são separadores de milhares
    value = value.replace(",", "")
  else: # Se uma vírgula, assume que é um separador decimal
    value = value.replace(",", ".")

  return float(value)

# Função para transformar strings de volume em inteiros, tratando sufixos 'M' e 'K'
def transforming_float(value):
  value = value.strip()
  if "m" in value.lower(): # Se 'm' (milhão) estiver presente
    # Tratamento especial: se a parte numérica (após remover 'm') for muito grande (>=10000),
    # é multiplicada por 10e3 (10.000) em vez de 10e6 (1.000.000). Isso parece incomum.
    if remove_letter(value) >= 10000:
      return int(remove_letter(value) * 10**3)
    return int(remove_letter(value) * 10**6)

  return int(remove_letter(value))


df1["Volume"] = df1["Volume"].apply(transforming_float) # Aplica a transformação à coluna 'Volume'

5.  **Processamento de Split Ratio (Proporção de Desdobramento)**
    * Análise revela uma quantidade significativa de ruído (valores inválidos ou não padronizados) na coluna 'Split Ratio'.


In [17]:
df1['Split Ratio'] = df1['Split Ratio'].fillna('Split inválido').astype(str)  # Substitui NaN por 'Split inválido' e garante tipo string
get_non_numeric_characters(df1["Split Ratio"])

Split Ratio
{'v', 'T', 'd', 'i', 'e', 'J', 'F', 'Z', 'E', 'B', 'Y', 'X', ' ', 'l', 'H', 'G', 'R', 'W', 'P', 'D', 'o', 'L', 'K', 'N', 'á', ':', 'p', 'A', 'O', 'M', 't', 'V', 'U', 'n', 'Q', 'S', 'I', 'C'}


Corrigindo valores com uma letra no lugar de dois pontos (ex: 2a1 ou 3K2)

In [18]:
# Função para verificar se uma string de 3 caracteres é um split ratio potencialmente válido com erro no caractere central
def poss_valid_split(valor):
  return valor[0].isdigit() and valor[1] != ":" and valor[2].isdigit()

# Itera e corrige split ratios com separadores incorretos
for i, valor in df1["Split Ratio"].items():
    if len(valor) == 3:
        if poss_valid_split(valor): # Se corresponder ao padrão (ex: "2a1")
            df1.loc[i, "Split Ratio"] = valor[0] + ":" + valor[2] # Corrige para "2:1"

Entendendo como é a distribuição de valores válidos

In [19]:
# Função para verificar se um valor é um split ratio formatado corretamente (ex: "1:2")
def right_split(val):
    val = str(val)
    return len(val) == 3 and val[0].isdigit() and val[1] == ":" and val[2].isdigit()

poss = [] # Lista para armazenar split ratios válidos únicos encontrados

# Itera pelos valores de 'Split Ratio' para encontrar formatos válidos únicos
for _, val in df1["Split Ratio"].items():
    if pd.isna(val):
        continue

    if right_split(val) and val not in poss: # Se formato válido e ainda não encontrado
        poss.append(val)

print("Split Ratios válidos encontrados:", poss)

Split Ratios válidos encontrados: ['1:1', '3:2', '2:1']


Ajustando splits parciais (ex: x:2 ou 3:f)
Em vasos com o valor 1 no final foi preciso gerar aleatóriamente os valor do numerador para manter a proporção da distribução.

In [20]:
import random
# Função para ajustar split ratios parcialmente válidos ou ligeiramente incorretos
def ajust_parcial_splits(val):
    if val[0] == "2": # Se começa com '2' e precisa de correção (ex: "2:f")
        return val[:-1] + "1" # Assume que deveria ser "2:1"
    if val[0] == "3": # Se começa com '3' e precisa de correção (ex: "3:f")
        return val[:-1] + "2" # Assume que deveria ser "3:2"
    if val[2] == "1": # Se termina com '1' e precisa de correção (ex: "f:1")
        newv = random.choice([1, 2]) # Escolhe aleatoriamente 1 ou 2 para a primeira parte
        return f"{newv}:{val[2]}"
    return "Valor inválido" # Se nenhuma regra se aplicar, marca como inválido

# Itera por 'Split Ratio' para aplicar ajustes
for index, value in df1["Split Ratio"].items():
    if not value == "Valor inválido" and (not value[0].isdigit() or not value[2].isdigit()):
        df1.loc[index, "Split Ratio"] = ajust_parcial_splits(value)

Quantidade elevada de valores inválidos e aparentemetne irreculperáveis

In [21]:
# Conta as ocorrências de "Valor inválido"
invalid_count = df1["Split Ratio"].value_counts().get("Valor inválido", 0)
# Calcula e imprime a proporção de split ratios inválidos
print(f"Proporção de valores inválidos: {invalid_count/len(df1['Split Ratio']):.2%}")

Proporção de valores inválidos: 25.76%


### Eliminando coluna "Split Ratio":
- Por estar com muitos dados faltantes e irrecuperáveis pela formula ou qualuquer tipo de inferência.
- Além disso os valores da coluna são irrelevantes para a maioria das analises posteriores.

In [22]:
df1.drop(columns=["Split Ratio"], inplace=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   ID         200000 non-null  int64         
 1   Date       197970 non-null  datetime64[ns]
 2   Ticker     200000 non-null  object        
 3   Open       200000 non-null  float64       
 4   High       200000 non-null  float64       
 5   Low        200000 non-null  float64       
 6   Close      200000 non-null  float64       
 7   Adj Close  200000 non-null  float64       
 8   Volume     200000 non-null  int64         
 9   Dividend   200000 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(2), object(1)
memory usage: 15.3+ MB


### Compreendendo os valores na tabela e tratando outliers de forma eficiente

In [23]:
numeric_cols = df1.select_dtypes(include=np.number).columns
print(numeric_cols)
# Itera sobre as colunas numéricas e gera um boxplot para cada uma
for col in numeric_cols:
    fig = px.box(df1, y=col, title=f'Boxplot de {col}')
    fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [24]:
df1["Ticker"].describe()
ticker_counts = df1["Ticker"].value_counts()
#v_ticker_counts = ticker_counts[ticker_counts["count"] > 10]

fig = px.box(ticker_counts, title="Distribuição de Ticker")
fig.show()

In [25]:
fig = px.bar(ticker_counts, title="Distribuição de Ticker")
fig.show()

In [26]:
fig = px.box(df1, x="Date", title="Distribuição de Datas")
fig.show()
df1["Date"].describe()

Output hidden; open in https://colab.research.google.com to view.

In [27]:
#df1.to_csv('/content/drive/MyDrive/Projeto Dados CITI (Squad Dani)/Databases/stock_part1_cleaned.csv', index=False) Comentado para não alterar o datafarme

# Verificação de tipos de dados e correções
Entendemos as características dos dados, identificamos problemas e priorizamos as váriaveis mais relevantes
-  Conversão Personalizada de Tipos com corrigir_tipos_personalizado() com o objetivo de padronizar os tipos de dados para cada coluna com base em seu conteúdo.

Regras de Conversão:

Colunas com nome date ou original_date: Convertidas para datetime.

Coluna ticker: Convertida para string.

Demais colunas: Tentativa de conversão para número decimal (float).

Tratamento de Erros: Utiliza errors='coerce' para garantir que entradas inválidas virem NaN, evitando falhas na execução.

- Diagnóstico Final com o objetivo de verificar se os dados foram corretamente padronizados.

Ações:

Exibe os tipos finais das colunas.

Mostra as primeiras linhas do DataFrame para inspeção visual rápida.


id                      float64;

date             datetime64[ns];

ticker                   object;

open                    float64;

high                    float64;

low                     float64;

close                   float64;

adj_close               float64;

volume                  float64;

dividend                float64;



original_date    datetime64[ns];

# **Stock Market Data Cleaning Pipeline**
Pipeline de limpeza e preparação de dados para análise financeira do dataset `stock_part2.csv.`

---

## **Principais Etapas**
### **Market Cap, PE Ratio, EPS e Previous Close**

* Remoção de caracteres especiais ($, ,, Z → 2).

* Conversão para float:

 Market Cap foi convertido para `dólares americanos`, com ajuste de unidade:

  * K → ×10³

  * M → ×10⁶

  * B → ×10⁹

  * T → ×10¹²

* Colunas numéricas (PE Ratio, EPS, Previous Close) `padronizadas para valores float.`

### **Next Earnings Date**
* Conversão de strings para o formato de `data` (datetime.date), com tentativa de múltiplos formatos (%d/%m/%Y, %Y-%m-%d, etc).

* Identificação de datas inválidas, substituídas por `'Data Inválida'.`

* Padronização final no formato `DD/MM/AAAA.`

### **Day Range**
* Separação da coluna Day Range em `duas colunas distintas:`

  * Day Low: Menor valor
  * Day Right: Maior Valor

* Remoção de caracteres não numéricos e `conversão para float.`

* Tratamento de casos inválidos com preenchimento `NaN.`

### **Currency**
* Padronização de todas as moedas para `'USD'`, através de:
  * Conversão para caixa alta.
  * Tratamento de múltiplas representações ('Cifrão', 'USDcifrão', 'USD', 'DOLLAR', etc).
  * Casos ausentes ou vazios também padronizados como 'USD'.

### **Expotação Final**
* Seleção das colunas principais:
  * ID, Sector, Industry, Exchange, Currency_std.
  * Variáveis financeiras limpas: Market Cap (USD), PE Ratio, EPS (USD), Previous Close (USD).
  * Informações sobre o próximo resultado: Next Earnings Date.
  * Faixa diária de preço: Day Low (USD), Day High (USD).

* Renomeação das colunas para formatos mais claros e padronizados.

* Exportação para arquivo CSV codificado em UTF-8.

## Imports e Configurações

Nesta seção, importamos as bibliotecas necessárias e definimos algumas configurações iniciais.


In [29]:
import re            # Para usar expressões regulares (manipular textos)
from datetime import datetime, date  # Para trabalhar com datas

## Leitura do DataFrame

Aqui fazemos a leitura do arquivo CSV base.

In [30]:
df = pd.read_csv('/content/drive/MyDrive/Projeto Dados CITI (Squad Dani)/Databases/stock_part2.csv', dtype=str)

## **Função de Limpeza e Transformação**

### **Coluna EPS (Lucro por Ação):**
* Processo similar ao clean_numeric, com ajustes adicionais:
  * Correção de 'Z' → '2' e vírgulas → pontos.

  * Remoção de caracteres não numéricos.

* Validação de integridade:
  * Se o valor convertido for muito grande (abs(valor) > 1000), considera como inválido e substitui por NaN.
  * Caso contrário, o valor é arredondado para 4 casas decimais para manter a precisão financeira.

In [31]:
# Função para tratar da coluna EPS
def clean_eps(value):
    if pd.isnull(value):
        return np.nan

    value = str(value).strip()

    value = clean_and_convert_number(value)

    if not value:
        return np.nan

    try:
        num = value
        if abs(num) > 1000:  # Se o valor for muito grande, considera inválido
            return np.nan
        return round(num, 4)  # Arredonda para 4 casas
    except:
        return np.nan

## **MarketCap - Valor de Mercado da Empresa**

**Verificação de Valores Nulos:**
  * Tratamento de Valores Nulos, retornando NaN caso estejam ausentes ou inválidos.

**Limpeza do valor:**

  * Remove aspas simples e duplas.

  * Elimina espaços extras.

  * Corrige possíveis erros de digitação trocando 'Z' ou 'z' por '2'.

  * Remove vírgulas, deixando o número puro.

**Separação entre valor numérico e unidade:**

  * Valor numérico: mantém apenas dígitos, pontos e sinais.

  * Unidade: mantém apenas letras, convertendo para maiúsculas.

**Conversão numérica:**

  * Tenta converter o valor para float.

  * Se não for possível, retorna NaN.

**Ajuste conforme a unidade:**

  * Se a unidade indicar:

    * Bilhões → multiplica por 1e9

    * Milhões → multiplica por 1e6

    * Trilhões → multiplica por 1e12

    * Milhares → multiplica por 1e3

Se não houver unidade reconhecida, mantém o valor como está.

In [32]:
def parse_marketcap(value):
    # Se o valor for nulo (faltando), retorna um valor padrão indicando ausência de dado
    if pd.isnull(value):
        return np.nan

    # Converte o valor para string, remove aspas e espaços extras
    val = str(value).replace('"', '').replace("'", '').strip()

    # Corrige o caractere 'Z' ou 'z' para '2', pois pode ter sido digitado errado
    val = val.replace('Z', '2').replace('z', '2')

    # Remove vírgulas e espaços, deixando o número "puro"
    val = val.replace(',', '').replace(' ', '')

    # Separa a parte numérica (ex.: "1.2") e a unidade (ex: "B" para bilhão)
    num_part = re.sub(r'[^\d\.\-]', '', val)  # Remove tudo que não for número, ponto ou hífen
    unit = re.sub(r'[\d\.\-]', '', val).upper()  # Remove números e transforma em maiúsculo

    try:
        # Tenta converter a parte numérica para um número de ponto flutuante
        num = float(num_part)
    except:
        # Se não conseguir, retorna nulo
        return np.nan

    # Agora ajusta o número conforme a unidade:
    # Se for bilhão (B), multiplica por 1 bilhão
    if 'B' in unit or 'BILLION' in unit:
        return num * 1e9
    # Se for milhão (M), multiplica por 1 milhão
    elif 'M' in unit or 'MILLION' in unit:
        return num * 1e6
    # Se for trilhão (T), multiplica por 1 trilhão
    elif 'T' in unit or 'TRILLION' in unit:
        return num * 1e12
    # Se for mil (K), multiplica por mil
    elif 'K' in unit or 'THOUSAND' in unit:
        return num * 1e3
    else:
        # Se não tiver nenhuma unidade reconhecida, retorna apenas o número como está
        return num

## **Função para processar intervalo do dia (Day Range)**

Extrair e padronizar valores mínimos e máximos de um intervalo diário representado como string, retornando sempre uma tupla ordenada (mínimo, máximo)

**Tratamento de valores ausentes:**

  * Se o valor for nulo (NaN), retorna uma tupla (np.nan, np.nan), indicando ausência de dados.

**Limpeza inicial:**

  * Converte o valor para string.

* Remove aspas e espaços extras.

**Separação dos valores:**

* Tenta separar a string com base em hífen ou traço (- ou –), símbolos comuns para indicar intervalos.

* Caso padrão — separação bem-sucedida:

* Se conseguir dividir em duas partes:

    * Remove tudo que não for número, ponto ou sinal negativo.

    * Converte cada parte para float.

    * Retorna como tupla ordenada: (menor valor, maior valor), usando min e max.

**Caso alternativo — separação falhou:**

  * Se não houver traço, busca identificar diretamente números na string.

* Se encontrar pelo menos dois números:

* Retorna os dois primeiros convertidos para float.

* Se não encontrar ou houver erro → retorna (np.nan, np.nan).

In [33]:
def parse_day_range(day_range):
    # Se o valor for nulo (ausente), retorna duas vezes "NaN" (não numérico)
    if pd.isnull(day_range):
        return (np.nan, np.nan)

    # Converte para string, remove aspas e espaços extras
    day_range = str(day_range).replace('"', '').replace("'", '').strip()

    # Divide a string com base em traços comuns (hífen '-' ou '–')
    parts = re.split(r'[-–]', day_range)

    # Se conseguiu dividir em duas partes (esperado: mínimo e máximo)
    if len(parts) == 2:

        try:
            # Remove qualquer caractere que não seja número, ponto ou sinal negativo
            low = float(re.sub(r'[^\d\.\-]', '', parts[0]))   # valor mínimo
            high = float(re.sub(r'[^\d\.\-]', '', parts[1]))  # valor máximo
            return min(low, high), max(low, high)  # Retorna como uma tupla (mínimo, máximo)
        except:
            # Se der erro na conversão, retorna duas vezes NaN
            return np.nan, np.nan
    else:
        # Se não conseguiu dividir com traço, tenta encontrar números diretamente

        nums = re.findall(r'[\d\.\-]+', day_range)

        if len(nums) >= 2:
            try:
                # Pega os dois primeiros números encontrados
                return float(nums[0]), float(nums[1])
            except:
                # Se der erro, retorna NaN
                return np.nan, np.nan
        else:
            # Se não encontrou pelo menos dois números, retorna NaN
            return np.nan, np.nan


## **Função para padronizar a moeda (Currency)**

Padronizar os valores da coluna de moeda (Currency), convertendo diferentes variações e abreviações para o formato padrão: "USD" (Dólar Americano).

**Tratamento de valores ausentes:**

* Se o valor for nulo (NaN) ou uma string vazia, assume "USD" como valor padrão.

**Normalização da entrada:**

* Converte a entrada para string, coloca tudo em maiúsculo e remove espaços.

**Casos comuns tratados:**

* Se for um dos símbolos comuns:
    * ['$', 'US$', 'U$'] → padroniza para "USD".

  * Se contiver "USD" em qualquer parte → padroniza como "USD".

* Se começar com "US" ou contiver variações associadas ao dólar, como:
    * "DOLLAR", "DOLAR", "DOL", "DLR" → padroniza para "USD".

* Se for uma sigla de 3 caracteres e tiver o padrão típico:
    * primeira letra "U" e última "D" → padroniza como "USD"
(exemplo: "UZD" → "USD").

**Padrão de segurança:**

Se nenhuma das verificações anteriores for satisfeita → retorna "USD" por segurança e consistência.

In [34]:
def standardize_currency(currency):
    # Se o valor for ausente (nulo) ou uma string vazia, retorna 'USD' como padrão
    if pd.isna(currency) or currency == '':
        return 'USD'

    # Converte o valor para string, coloca tudo em maiúsculo e remove espaços extras
    currency_str = str(currency).upper().strip()

    # Caso o valor seja um dos símbolos comuns para dólar, ou contenha 'USD', padroniza como 'USD'
    if currency_str in ['$', 'US$', 'U$'] or 'USD' in currency_str:
        return 'USD'

    # Se começar com 'US', ou contiver palavras relacionadas ao dólar, também padroniza como 'USD'
    if (currency_str.startswith('US') or
        'DOLLAR' in currency_str or
        'DOLAR' in currency_str or
        'DOL' in currency_str or
        'DLR' in currency_str):
        return 'USD'

    # Se tiver exatamente 3 caracteres e for algo como 'USD', também padroniza
    if len(currency_str) == 3 and currency_str[0] == 'U' and currency_str[2] == 'D':
        return 'USD'

    # Por fim, se não se encaixar em nenhuma das verificações, retorna 'USD' como padrão
    return 'USD'


In [35]:
corrections = {}
rare_valid = 0
df["Exchange"] = df["Exchange"].str.upper().str.strip()
exchange_counts = df["Exchange"].value_counts()

valid_exchangers = exchange_counts[exchange_counts > 52].index.tolist()

for ex in exchange_counts[exchange_counts <= 52].index:

  corrected = get_fuzzy_match(ex, valid_exchangers)

  if corrected:
    corrections[ex] = corrected
  else:
    #df.drop(df[df["Ticker"] == ticker].index, inplace=True)
    print(f"Não foi possivel encontrar uma valor parecido com {ex}")

df["Exchange"] = df["Exchange"].replace(corrections)
print("corrections:", len(corrections))

corrections: 868


In [36]:
valid_exchangers

['HKEX', 'TSX', 'SSE', 'EURONEXT', 'NYSE', 'NASDAQ', 'AMEX', 'TSE', 'LSE']

## Processamento e Limpeza das Colunas

Aplicamos as funções criadas anteriormente para limpar e padronizar as colunas do DataFrame.

In [37]:
# Verificação das colunas na planilha e atualizações
df['MarketCap_clean'] = df['MarketCap'].apply(parse_marketcap) if 'MarketCap' in df.columns else np.nan

df['PE Ratio_clean'] = df['PE Ratio'].apply(clean_and_convert_number) if 'PE Ratio' in df.columns else np.nan
df['EPS_clean'] = df['EPS'].apply(clean_eps) if 'EPS' in df.columns else np.nan

df['Previous Close_clean'] = df['Previous Close'].apply(clean_and_convert_number) if 'Previous Close' in df.columns else np.nan

df, invalid_dates, future_dates, error_percent = process_date_column(df, 'Next Earnings')
df.rename(columns={'Next Earnings': 'Next Earnings Date'}, inplace=True)
df['Next Earnings_str'] = df['Next Earnings Date'].apply(
        lambda x: x.strftime('%d/%m/%Y') if pd.notnull(x) else 'Data inválida'
    )

if 'Day Range' in df.columns:
    # Aplica a função 'parse_day_range' para separar em duas colunas: mínima e máxima do dia.
    # O 'apply(pd.Series)' expande a tupla retornada em duas colunas.
    df[['Day Low', 'Day High']] = df['Day Range'].apply(parse_day_range).apply(pd.Series)
else:
    df['Day Low'], df['Day High'] = np.nan, np.nan
df['Currency_std'] = df['Currency'].apply(standardize_currency) if 'Currency' in df.columns else 'USD'


Percentage of errors in 'Next Earnings': 1.00%


In [38]:
df["Currency_std"].value_counts()

,count
Currency_std,
USD,200000


In [39]:
df

,ID,Sector,Industry,Exchange,Currency,MarketCap,PE Ratio,EPS,Next Earnings Date,Previous Close,Day Range,MarketCap_clean,PE Ratio_clean,EPS_clean,Previous Close_clean,Next Earnings_str,Day Low,Day High,Currency_std
0,1,TELECOM,Chemicals,NYSE,USD,"370,815,220,190",27.40,16.68,2025-05-19,567.57,X095.58-898.36,3.708152e+11,27.40,16.68,567.57,19/05/2025,95.58,898.36,USD
1,2,Real Estate,Property Mgmt,TSE,$,1289775810656,90.13,"0,57",2025-06-23,$185.1Z,907.52-552.85,1.289776e+12,90.13,0.57,185.12,23/06/2025,552.85,907.52,USD
2,3,healthcare,Biotech,LSE,$,97815908000,$15.67,18.17,2025-07-21,823.07,1380.78-1361.69,9.781591e+10,15.67,18.17,823.07,21/07/2025,1361.69,1380.78,USD
3,4,Telecom,Electric Power,HKEX,$,1658573090566,83.90,-4.82,2025-07-23,179.88,457.99-138.11,1.658573e+12,83.90,-4.82,179.88,23/07/2025,138.11,457.99,USD
4,5,retail,Banking,NASDAQ,US DOLLAR,315341356874,79.86,5.67,2025-08-06,$1004.79,644.16-920.32,3.153414e+11,79.86,5.67,1004.79,06/08/2025,644.16,920.32,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199996,Telecom,Oil & Gas,EURONEXT,USD,1896704951586,94.38,15.10,2025-05-22,749.55,1486.17-538.54,1.896705e+12,94.38,15.10,749.55,22/05/2025,538.54,1486.17,USD
199996,199997,Real Estate,BiotecG,TSE,$,319027664163,61.69,"2,69",2025-06-14,376.85,1275.46-1239.11,3.190277e+11,61.69,2.69,376.85,14/06/2025,1239.11,1275.46,USD
199997,199998,Real Estate,Wireless Telecom,TSX,USD,822395576541,30.42,"3,75",2025-07-14,1084.33,1479.14-565.90,8.223956e+11,30.42,3.75,1084.33,14/07/2025,565.90,1479.14,USD
199998,199999,technology,Chemicals,EURONEXT,$,134796316185,63.49,19.86,2025-08-04,595.77,1317.70-646.91,1.347963e+11,63.49,19.86,595.77,04/08/2025,646.91,1317.70,USD


## Exportação do Arquivo Final

Selecionamos e renomeamos as colunas finais, exportando o DataFrame limpo para um arquivo CSV.


In [40]:
# Seleciona as colunas desejadas do DataFrame 'df' para criar um novo DataFrame 'output_data'.
# Essas colunas são as principais informações limpas e padronizadas que queremos exportar.
output_data = df[[
    'ID',                 # Identificador único de cada ação.
    'Sector',             # Setor da empresa.
    'Industry',           # Indústria da empresa.
    'Exchange',           # Bolsa de valores onde a ação é negociada.
    'MarketCap_clean',    # Valor de mercado limpo, convertido para número.
    'PE Ratio_clean',     # Relação Preço/Lucro limpa.
    'EPS_clean',          # Lucro por ação limpo.
    'Next Earnings_str',  # Próxima data de divulgação de resultados formatada.
    'Previous Close_clean', # Preço de fechamento anterior limpo.
    'Day Low',            # Mínima do dia.
    'Day High'            # Máxima do dia.
]].rename(columns={
    # Renomeia as colunas para nomes mais legíveis e padronizados no arquivo final.
    'MarketCap_clean': 'Market Cap (USD)',
    'PE Ratio_clean': 'PE Ratio',
    'EPS_clean': 'EPS (USD)',
    'Next Earnings_str': 'Next Earnings Date',
    'Previous Close_clean': 'Previous Close (USD)',
    'Day Low': 'Day Low (USD)',
    'Day High': 'Day High (USD)'
})

# Substitui valores nulos ou inválidos na coluna 'Next Earnings Date' por 'Data Inválida'.
# Isso garante que o arquivo exportado fique mais legível e padronizado.
output_data['Next Earnings Date'] = output_data['Next Earnings Date'].replace({
    'NaT': 'Data Inválida',  # 'NaT' é um valor nulo para datas no pandas.
    None: 'Data Inválida'    # Também trata casos de valores None.
})

# Exporta o DataFrame limpo para um arquivo CSV.
"""output_data.to_csv(
    '/content/drive/MyDrive/Projeto Dados CITI (Squad Dani)/Databases/stock_part2_cleaned.csv',
    index=False,
    encoding='utf-8'
)"""

"output_data.to_csv(\n    '/content/drive/MyDrive/Projeto Dados CITI (Squad Dani)/Databases/stock_part2_cleaned.csv',\n    index=False,\n    encoding='utf-8'\n)"

# **Boxplot Referente ao tratamento do dataset stock_part2.csv.`**

In [42]:
df2 = pd.read_csv('/content/drive/MyDrive/Projeto Dados CITI (Squad Dani)/Databases/stock_part2_cleaned.csv')

# Identifica automaticamente as colunas numéricas
numeric_cols = df2.select_dtypes(include=np.number).columns
print("Colunas numéricas detectadas:", numeric_cols)

# Itera sobre as colunas numéricas e gera um boxplot para cada uma
for col in numeric_cols:
    fig = px.box(df2, y=col, title=f'Boxplot de {col}')
    fig.show()

Output hidden; open in https://colab.research.google.com to view.

Tipos de dados finais:
id                               int64;

sector                          object;

industry                        object;

exchange                        object;

currency_std                    object;

market cap (usd)                object;

pe ratio                       float64;

eps (usd)                      float64;

next earnings date      datetime64[ns];

previous close (usd)           float64;

day low (usd)                  float64;

day high (usd)                 float64;